In [1]:
# 测试基础导入（新增的测试单元格）
import os
import sys

print("Python:", sys.executable)
print("工作目录:", os.getcwd())

# 禁用可能导致卡住的代理
if 'all_proxy' in os.environ:
    print(f"检测到 SOCKS 代理: {os.environ['all_proxy']}")
    del os.environ['all_proxy']
    print("已临时禁用 SOCKS 代理")

print("\n环境准备完成，可以继续运行下一个单元格。")

Python: /bin/python
工作目录: /home/guci/aiProjects/mda/pim-compiler/react_is_all_you_need
检测到 SOCKS 代理: socks5://127.0.0.1:7890
已临时禁用 SOCKS 代理

环境准备完成，可以继续运行下一个单元格。


## 如何查看 Agent 的详细思维过程

### 1. 设置环境变量
```python
os.environ['LANGCHAIN_VERBOSE'] = 'true'
```

### 2. Agent 已经默认设置了 verbose=True
在 `react_agent.py` 的第 438 行：
```python
executor_config = {
    "agent": agent,
    "tools": tools,
    "verbose": True,  # 这里已经启用了详细输出
    "max_iterations": 50,
    "handle_parsing_errors": True
}
```

### 3. 当前输出显示的内容
- **工具调用决策**：显示 Agent 决定调用哪个工具及参数
- **工具执行结果**：显示每个工具的返回值
- **执行步骤**：显示 Agent 的执行流程

### 4. LangChain 的限制
- Agent 的内部"思考"（Chain of Thought）不会完全暴露
- 只能看到工具调用的决策和结果
- 这是 LangChain 设计的限制，不是配置问题

### 5. 如需更详细的输出
可以考虑：
- 修改 system prompt 让 Agent 解释其推理过程
- 使用 LangSmith 追踪（需要 API key）
- 添加自定义 callbacks 来拦截更多信息

## Agent Research Notebook - 使用 Moonshot (Kimi) API

本 notebook 展示如何使用 GenericReactAgent，并配置为使用 Moonshot (Kimi) API。

### API 配置信息
- **Base URL**: `https://api.moonshot.cn/v1`
- **Model**: `kimi-k2-0711-preview`（128k 上下文窗口）
- **API Key**: 需要设置环境变量 `MOONSHOT_API_KEY`

### 上下文窗口自动配置

系统会根据模型自动设置上下文窗口大小，并相应调整 memory 配置：

| 模型 | 上下文窗口 | 自动 memory 限制 |
|------|-----------|-----------------|
| kimi-k2-0711-preview | 128k | 102k (80%) |
| gpt-4-turbo | 128k | 102k (80%) |
| claude-3-opus | 200k | 160k (80%) |
| deepseek-chat | 32k | 25k (80%) |

### 设置 API Key 的方法

1. **使用 .env 文件（推荐）**：
   ```bash
   # 在项目根目录创建 .env 文件
   MOONSHOT_API_KEY=your_api_key_here
   ```

2. **使用环境变量**：
   ```bash
   export MOONSHOT_API_KEY=your_api_key_here
   ```

3. **在 notebook 中临时设置**：
   ```python
   import os
   os.environ['MOONSHOT_API_KEY'] = 'your_api_key_here'
   ```

### 其他支持的 LLM 配置示例

```python
# DeepSeek (默认)
config = ReactAgentConfig(
    work_dir="output",
    # 无需指定 LLM 参数，默认使用 DeepSeek
)

# OpenAI
config = ReactAgentConfig(
    work_dir="output",
    llm_model="gpt-4-turbo-preview",
    llm_base_url="https://api.openai.com/v1",
    llm_api_key_env="OPENAI_API_KEY"
)

# Claude (Anthropic)
config = ReactAgentConfig(
    work_dir="output",
    llm_model="claude-3-opus-20240229",
    llm_base_url="https://api.anthropic.com/v1",
    llm_api_key_env="ANTHROPIC_API_KEY"
)

# 手动指定上下文窗口
config = ReactAgentConfig(
    work_dir="output",
    llm_model="custom-model",
    llm_base_url="https://api.custom.com/v1",
    llm_api_key_env="CUSTOM_API_KEY",
    context_window=200000  # 手动指定 200k 上下文窗口
)
```

### Memory 级别详解

GenericReactAgent 支持三种记忆级别：

1. **NONE（无记忆）**
   - 每次对话都是独立的
   - 适合单次任务执行
   
2. **SMART（智能摘要缓冲）** - 使用 `ConversationSummaryBufferMemory`
   - 保留最近的对话原文直到达到 token 限制
   - 对超出限制的历史对话自动生成摘要
   - 适合长对话场景，既保留了重要信息又控制了 token 使用
   - 对于 Kimi K2 (128k)，可以保留约 100k tokens 的原始对话
   
3. **PRO（持久化存储）**
   - 使用 SQLite 数据库保存所有历史
   - 支持跨会话恢复对话
   - 适合需要长期记忆的项目

### ConversationSummaryBufferMemory 的优势

对于大上下文模型（如 Kimi K2 的 128k），使用 `ConversationSummaryBufferMemory` 的好处：

1. **智能管理历史**：自动决定哪些对话需要摘要，哪些保留原文
2. **最大化上下文利用**：充分利用 128k 的上下文窗口
3. **保留关键信息**：通过摘要保留早期对话的关键信息
4. **动态调整**：根据对话长度自动调整摘要策略

In [2]:
# 设置详细输出的环境变量示例
import os
import sys

# 保存原始代理设置
original_proxy = {
    'http_proxy': os.environ.get('http_proxy'),
    'https_proxy': os.environ.get('https_proxy'),
    'all_proxy': os.environ.get('all_proxy')
}

# 加载 .env 文件中的环境变量（如果使用 python-dotenv）
try:
    from dotenv import load_dotenv  # type: ignore
    load_dotenv()
except ImportError:
    print("提示：python-dotenv 未安装，跳过 .env 文件加载")
    print("可以使用 pip install python-dotenv 安装")

# 确保设置了 MOONSHOT_API_KEY
if not os.getenv('MOONSHOT_API_KEY'):
    print("警告：未设置 MOONSHOT_API_KEY 环境变量")
    print("请使用以下方式之一设置：")
    print("1. 创建 .env 文件并添加 MOONSHOT_API_KEY=your_api_key")
    print("2. 设置环境变量：export MOONSHOT_API_KEY=your_api_key")
    print("3. 在代码中设置：os.environ['MOONSHOT_API_KEY'] = 'your_api_key'")

# 方法1: 设置 LANGCHAIN_VERBOSE
# os.environ['LANGCHAIN_VERBOSE'] = 'true'

# 方法2: 设置日志级别
import logging
# logging.basicConfig(level=logging.DEBUG)
# logging.getLogger("langchain").setLevel(logging.DEBUG)

# 方法3: 使用 callbacks
try:
    from langchain_core.callbacks import StdOutCallbackHandler  # type: ignore
except ImportError:
    # 如果新的导入路径不存在，尝试旧路径
    try:
        from langchain.callbacks import StdOutCallbackHandler  # type: ignore
    except ImportError:
        print("提示：无法导入 StdOutCallbackHandler，某些功能可能不可用")

print("开始导入 react_agent...")
# 创建带 callback 的 agent
from react_agent import GenericReactAgent, ReactAgentConfig, MemoryLevel
print("导入完成!")

# 恢复代理设置（react_agent.py 可能会删除它们）
for key, value in original_proxy.items():
    if value:
        os.environ[key] = value
        
print("环境准备完成。")

开始导入 react_agent...
导入完成!
环境准备完成。


In [3]:
# 使用 Moonshot (Kimi) API 配置
# Kimi K2 模型有 128k 的上下文窗口，系统会自动调整 memory 配置
config = ReactAgentConfig(
    work_dir="output",
    memory_level=MemoryLevel.SMART,
    llm_model="kimi-k2-0711-preview",
    llm_base_url="https://api.moonshot.cn/v1",
    llm_api_key_env="MOONSHOT_API_KEY",
    llm_temperature=0
    # context_window 会自动设置为 131072 (128k)
    # max_token_limit 会自动调整为 104857 (上下文窗口的 80%)
)

agent = GenericReactAgent(config)

# 打印配置信息
print(f"模型: {config.llm_model}")
print(f"上下文窗口: {config.context_window} tokens")
print(f"内存 token 限制: {config.max_token_limit}")

task='''
你好，我的名字是谷词
'''

agent.execute_task(task)

模型: kimi-k2-0711-preview
上下文窗口: 131072 tokens
内存 token 限制: 104857

> Executing task...

👤 用户: 
你好，我的名字是谷词


💬 工具结果 (None):
   你好，谷词！很高兴认识你。我是你的AI助手，可以帮助你完成各种任务，比如：

- 代码开发和调试
- 项目架构设计
- 文档编写和分析
- 技术问题解答
- 文件和目录管理

有什么我可以帮助你的吗？

> Task completed.

=== 最终结果 ===

你好，谷词！很高兴认识你。我是你的AI助手，可以帮助你完成各种任务，比如：

- 代码开发和调试
- 项目架构设计
- 文档编写和分析
- 技术问题解答
- 文件和目录管理

有什么我可以帮助你的吗？


In [4]:
task='''
我的名字是什么？
'''

agent.execute_task(task)


> Executing task...

👤 用户: 
我的名字是什么？


💬 工具结果 (None):
   你的名字是**谷词**。

> Task completed.

=== 最终结果 ===

你的名字是**谷词**。


In [ ]:
# 测试 ConversationSummaryBufferMemory 的效果
# 这个 memory 会智能管理对话历史

# 创建一个新的 agent 实例来测试长对话
test_config = ReactAgentConfig(
    work_dir="output/memory_test",
    memory_level=MemoryLevel.SMART,
    llm_model="kimi-k2-0711-preview",
    llm_base_url="https://api.moonshot.cn/v1",
    llm_api_key_env="MOONSHOT_API_KEY",
    llm_temperature=0
)

test_agent = GenericReactAgent(test_config)

# 模拟多轮对话
print("=== 测试 ConversationSummaryBufferMemory ===\n")

# 第一轮
test_agent.execute_task("我叫张三，我是一名软件工程师")

# 第二轮  
test_agent.execute_task("我最喜欢的编程语言是Python")

# 第三轮
test_agent.execute_task("我的名字是什么？我的职业是什么？")

# 随着对话继续，早期的对话会被自动摘要
# 但重要信息（如名字、职业）会被保留

In [ ]:
# 测试 LangChain 缓存效果
# temperature=0 时，相同的输入会使用缓存

import time

# 第一次调用（会调用 API）
start = time.time()
agent.execute_task("什么是 Python？")
first_time = time.time() - start
print(f"\n第一次调用耗时: {first_time:.2f} 秒")

# 第二次调用相同问题（应该使用缓存）
start = time.time()
agent.execute_task("什么是 Python？")
second_time = time.time() - start
print(f"\n第二次调用耗时: {second_time:.2f} 秒")

print(f"\n缓存加速: {first_time/second_time:.1f}x")

In [6]:
# 创建配置 - 使用 Moonshot API
config = ReactAgentConfig(
    work_dir="output",
    memory_level=MemoryLevel.NONE,
    # knowledge_file="knowledge/综合知识.md",
    llm_model="kimi-k2-0711-preview",
    llm_base_url="https://api.moonshot.cn/v1",
    llm_api_key_env="MOONSHOT_API_KEY",
    llm_temperature=0
)

# 创建 Agent
agent = GenericReactAgent(config)

# 执行任务
task="项目根目录是/home/guci/aiProjects/mda/pim-compiler，他的体系架构和核心组件是什么？"
# task="编程规范和项目结构知识是什么？"
agent.execute_task(task)


> Executing task...

👤 用户: 项目根目录是/home/guci/aiProjects/mda/pim-compiler，他的体系架构和核心组件是什么？

🤔 AI 思考: 需要使用工具来完成任务
🔧 调用工具: list_directory
   参数: {'directory_path': '/home/guci/aiProjects/mda/pim-compiler'}

💬 工具结果 (list_directory):
   [DIR] .benchmarks
[DIR] .cache
[FILE] .coverage.guci-Latitude-E5470.3273041.XGtwSYEx
[FILE] .env
[FILE] .env.example
[DIR] .github
[FILE] .gitignore
[FILE] .langchain.db
[FILE] .langchain_cache.db
[FILE] .langchain_test_cache.db
[DIR] .mypy_cache
[DIR] .pim_cache
[DIR] .pytest_cache
[DIR] .vscode
[FILE] ARCHITECTURE_CHANGES.md
[FILE] CHANGELOG.md
[FILE] ENHANCED_FEATURES.md
[FILE] GEMINI_KNOWLEDGE.md
[FILE] GENERATOR_COMPARISON.md
[FILE] GENERATOR_GUIDE.md
[FILE] GENERATOR_TEST_RESULTS.md
[FILE]...
   [省略 8663 字符]

🤔 AI 思考: 需要使用工具来完成任务
🔧 调用工具: read_file
   参数: {'file_path': '/home/guci/aiProjects/mda/pim-compiler/README.md'}

💬 工具结果 (read_file):
   # PIM Compiler v3.1

将平台无关模型（PIM）自动转换为可执行代码的智能编译器，完全基于 Gemini CLI 实现。

## 特性

- 🚀 **一键编译**：从业务模型直接生成可运行的代码
- 🤖 *

In [ ]:
# 创建配置 - 使用 Moonshot API
config = ReactAgentConfig(
    work_dir="output",
    memory_level=MemoryLevel.NONE,
    knowledge_file="pim_to_psm_knowledge.md",
    llm_model="kimi-k2-0711-preview",
    llm_base_url="https://api.moonshot.cn/v1",
    llm_api_key_env="MOONSHOT_API_KEY",
    llm_temperature=0
)

# 创建 Agent
psm_expert = GenericReactAgent(config)
psm_expert.specification="psm专家，擅长将pim文档转化成psm文档"

# 执行任务
psm_expert.execute_task("/home/guci/aiProjects/mda/pim-compiler/examples/blog.md  转化成psm保存在/home/guci/aiProjects/mda/pim-compiler/examples/blog_psm.md")

### 使用 Google 搜索功能

GenericReactAgent 现在支持 Google 搜索和网页读取功能。需要先设置 Google API：

1. 获取 Google API Key: https://console.cloud.google.com/apis/credentials
2. 创建自定义搜索引擎: https://programmablesearchengine.google.com/
3. 在 .env 文件中设置：
   ```
   GOOGLE_API_KEY=your_api_key_here
   GOOGLE_CSE_ID=your_cse_id_here
   ```

如果需要代理访问 Google，还需设置：
```
HTTP_PROXY=http://your_proxy:port
```

In [ ]:
# 测试搜索功能的 Agent
# 注意：需要先设置 Google API 凭证

# 创建带搜索功能的 Agent
search_config = ReactAgentConfig(
    work_dir="output/search_test",
    memory_level=MemoryLevel.SMART,
    llm_model="kimi-k2-0711-preview",
    llm_base_url="https://api.moonshot.cn/v1",
    llm_api_key_env="MOONSHOT_API_KEY",
    llm_temperature=0
)

search_agent = GenericReactAgent(search_config)

# 测试搜索功能
task = """
请帮我网络搜索关于 "LangChain memory 类型" 的信息，并总结不同类型的 memory 的特点。
"""

search_agent.execute_task(task)